## Remember to save this as a new notebook before you begin solving!!
## Also remember to open the notebook through a virtual env that works well with keras

### This exercise is meant to teach you how to a powerful, existing neural network for a new problem in a similar domain. This exercise should come after you have some experience with NN (not necessarily extensive experience)

The core of the exercise is downloading a trained NN for face classification, and using transfer learning techniques to solve a new problem. The new problem you will solve is classification of faces - automatic labeling of the different members of your family.

### Authors: Philip Tannor, Gal Eyal

# Creating your data 

In this exercise, you will get your hands dirty and create your own data:
1. Start by creating a folder with many photos of yourself (between 100 and 300).
2. Create similar folders for a number of roommates or family members (preferably people who live with you). Try to make sure the folders have similar amounts of photos in them.
3. Use the python program cropall.py (*works on linux only!!*), which should be in the folder of this exercise, to crop out the faces from the different photos. You should create a x/y ratio of 3/4, don't demand a perfect pixel ratio, and change the x/y ratio to 4/3 when you have a rotated photo. I you're working correctly, you should only be using the mouse click, the mouse scroll, and Spacebar.
4. Make sure to save the cropped photos of each person in a different folder. Notice that by default the crops will all go to the same folder, so find a way to deal with this manually.

# Imports and parameters

We left this section almost exactly as it was were when we created the solution, just to save some time. Don't feel bonded by these libraries (or parameters).

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
import numpy as np
#These next 2 imports depend on the 'keras-vggface' folder in https://github.com/rcmalli/keras-vggface
#Just in case the git won't be around, we put this folder in the directory you're working in.
#Notice this if you move the notebook around.
from keras_vggface import VGGFace
from keras_vggface import utils
from keras.preprocessing import image
import keras
import unittest
import pandas as pd
from os import listdir,path
from sklearn.model_selection import train_test_split
#This next import will help you with augmentation - generating augmented photos from your originals.
#Read about this general teqnique, and also about 
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense
from keras.models import Model

In [3]:
input_size=(224, 224)
n_channels = 3

In [4]:
keras.backend.set_image_dim_ordering('tf')

## Try testing one of the existing models.
1. Create an instance of the VGGFace class you imported, and chose which model it will be using.
2. Use keras.preprocessing.image to read the only picture in the 'image' file (fix the target size).
3. Use vgg-keras.utils.preprocess_input for preprocessing the photo. You may have to expand the dimensions before this.
4. Use the model to create a prediction for the image.
5. Check how the model did. The guy you looked at should be indexed first.
6. Restart the kernel and next time skip this section (so that you aren't wasting RAM).

In [5]:
model = VGGFace(model='vgg16')
img = image.load_img('image/ajb.jpg', target_size=input_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = utils.preprocess_input(x, version=1)
preds = model.predict(x)
plt.plot(preds[0,:], '.')
plt.show()
preds[0,:]
utils.decode_predictions(preds)

## Reading and preprocessing the data you created
create a DataFrame with 2 columns:
1. The exact path of each image
2. The label of each image

Split this data into train and test sets

In [6]:
df = pd.DataFrame(columns=['image_path', 'label'])
for folder in listdir('./data'):
    for im in listdir(path.join('./data', folder)):
        df = df.append({'image_path': path.join('./data', folder, im),
                  'label': folder}, ignore_index=True)

In [7]:
train, test = train_test_split(df, test_size=0.2)

Now use iterate through the dataframes and read the images corresponding to each row (using keras.preprocessing.image). Save the images as numpy arrays, and once again use utils.preprocess_input to preprocess the arrays.
Save the labels in 2 different ways:
1. Onehot encoding - for using a NN
2. Serial number of the corresponding class (one number per label) - for using an SVM

In [8]:
train_x = np.zeros((len(train), input_size[0], input_size[1], n_channels))
for i, row in enumerate(train.itertuples()):
    img = image.load_img(row.image_path, target_size=input_size)
    x = image.img_to_array(img)
    train_x[i, :] = x
train_x = utils.preprocess_input(train_x, version=1)
train_y = pd.get_dummies(train['label'])

In [10]:
test_x = np.zeros((len(test), input_size[0], input_size[1], n_channels))
for i, row in enumerate(test.itertuples()):
    img = image.load_img(row.image_path, target_size=input_size)
    x = image.img_to_array(img)
    test_x[i, :] = x
test_x = utils.preprocess_input(test_x, version=1)
test_y = pd.get_dummies(test['label'])

Create an instance of keras.preprocessing.image.ImageDataGenerator, which will define how you will create augmentations of each original image you've created. Chose all the parameters on your own.

In [12]:
train_gen = ImageDataGenerator(rotation_range=90, width_shift_range=0.1, height_shift_range=0.1,zoom_range=0.2, horizontal_flip=True)

# Finally - transfer learning. 
### This should be done with the keras functional API (not Sequential)
1. Create an instance of VGGFace with the model of your choice. 
2. Examine the model's architechture using .summary(). Understand what it means to replace the last 2 dense layers (including the final softmax layer). 
3. Use .get_layer() to retrieve the last layer which you want to keep in your new NN.
4. Create the 2 new Dense layers which continue the previous pretrained layers (the last layer should have a softmax activation).
5. Create a new model which the input of the original model as input, and outputs the new dense-softmax layer. You can use .input on the old model for this.
6. Freeze all of the layers except the last 2 using .layers on the new model, and .trainable = False. This will stop you from training those layers.
7. Compile the model ('adam' worked ok for us as an optimizer).

In [13]:
#custom parameters
nb_class = 2
hidden_dim = 512

vgg_model = VGGFace(input_shape=(224, 224, 3))
dense_1 = vgg_model.get_layer('fc6/relu').output
#new_dense_1 = Dense(hidden_dim, activation='relu', name='fc6')(flatten)
new_dense_1 = Dense(hidden_dim, activation='relu', name='fc7')(dense_1)
out = Dense(nb_class, activation='softmax', name='fc8')(new_dense_1)
custom_vgg_model = Model(vgg_model.input, out)

In [14]:
for layer in custom_vgg_model.layers[:-2]:
    layer.trainable = False

In [18]:
custom_vgg_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

Now you're ready to train the model:
1. Use .fit_generator() and not .fit(), since you'll be using the augmentor you created.
2. Use .flow() on the instance of ImageDataGenerator as the first input. 
3. Choose a combination on batch_size(within .flow) and steps_per_epoch which will create a total number of images that you want per each epoch. 
4. Use validation data which isn't augmented.

How well is it doing? We got an accuracy of 90%-95% percent (validation data) on photos on the 2 of us (just 2 classes).

In [19]:
custom_vgg_model.fit_generator(train_gen.flow(train_x, train_y, batch_size=20, seed=77), \
        steps_per_epoch=13, epochs=100, validation_data=(test_x, test_y), workers=50)

Epoch 1/100
13/13 [==============================] - 36s 3s/step - loss: 3.4984 - acc: 0.7247 - val_loss: 1.0593 - val_acc: 0.9091
Epoch 2/100
13/13 [==============================] - 15s 1s/step - loss: 2.1545 - acc: 0.8305 - val_loss: 0.9808 - val_acc: 0.9273
Epoch 3/100
13/13 [==============================] - 15s 1s/step - loss: 1.8975 - acc: 0.8538 - val_loss: 1.0515 - val_acc: 0.9273
Epoch 4/100
13/13 [==============================] - 15s 1s/step - loss: 1.8186 - acc: 0.8731 - val_loss: 1.6155 - val_acc: 0.8909
Epoch 5/100
13/13 [==============================] - 15s 1s/step - loss: 2.1879 - acc: 0.8486 - val_loss: 1.4573 - val_acc: 0.9091
Epoch 6/100
13/13 [==============================] - 15s 1s/step - loss: 2.1815 - acc: 0.8498 - val_loss: 1.1658 - val_acc: 0.9273
Epoch 7/100
13/13 [==============================] - 15s 1s/step - loss: 1.6433 - acc: 0.8844 - val_loss: 1.1775 - val_acc: 0.9273
Epoch 8/100
13/13 [==============================] - 15s 1s/step - loss: 2.8150 - a

KeyboardInterrupt: 

# Similar exercise with SVM
1. Now create a new model which outputs the data from the last frozen layer. Use this model to create feature for each image.
2. Create a SVM classifier using these features. We used sklearn.svm.SVC.
3. Did this work better or less good? Evaluate using sklearn.metrics.classification_report or sklearn.metrics.accuracy_score.
4. Try the same thing with the one-before-last layer.

In [21]:
final_dense = vgg_model.get_layer('fc7/relu').output
cnn_code_model = Model(vgg_model.input, final_dense)
train_f = cnn_code_model.predict(train_x)
test_f = cnn_code_model.predict(test_x)

In [28]:
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(train_f,train_y.gal)
test_p=svm_model.predict(test_f)

In [40]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [39]:
print (classification_report(test_y.gal, test_p))

             precision    recall  f1-score   support

          0       0.87      0.96      0.91        27
          1       0.96      0.86      0.91        28

avg / total       0.91      0.91      0.91        55



In [41]:
accuracy_score(test_y.gal, test_p)

0.9090909090909091

# Congrats! You completed the exercise. 
# Bonus - use one of these models and a webcam to created a personalized security system for your house.

In [1]:
scores = {'open_cargo_area': 0.76805175603205811, 'sunroof': 0.86970924721930554, 'yellow': 0.89589529966763537, 'ac_vents': 0.96354515221579118, 'large vehicle': 0.967633060324956, 'soft_shell_box': 0.12349743597877082, 'small vehicle': 0.99923983599401911, 'flatbed': 0.41886243386243383, 'blue': 0.60881424782853055, 'van': 0.74821960084759509, 'wrecked': 0.30877492983981797, 'crane truck': 0.73290598290598297, 'harnessed_to_a_cart': 0.11041046723608668, 'pickup': 0.63983575751303001, 'other': 0.10624473746628872, 'black': 0.87419993496329729, 'enclosed_cab': 0.29070354292695971, 'light truck': 0.46831705714546018, 'green': 0.047688623423157162, 'white': 0.93120701947835438, 'ladder': 0.020864457138074614, 'red': 0.83955850703367685, 'prime mover': 0.66474348026280405, 'bus': 0.96542111400274411, 'dedicated agricultural vehicle': 0.53234126984126984, 'enclosed_box': 0.64633525361595334, 'cement mixer': 0.17111823555768785, 'sedan': 0.96088493835027711, 'hatchback': 0.81490439599803766, 'minibus': 0.36037590386145602, 'jeep': 0.12514606086779878, 'silver/grey': 0.806923012930991, 'spare_wheel': 0.079828318098639134, 'truck': 0.55628477043330549, 'minivan': 0.44816745345990783, 'luggage_carrier': 0.80062166303241111, 'tanker': 0.08553744730215318}

In [6]:
scores

{'ac_vents': 0.9635451522157912,
 'black': 0.8741999349632973,
 'blue': 0.6088142478285306,
 'bus': 0.9654211140027441,
 'cement mixer': 0.17111823555768785,
 'crane truck': 0.732905982905983,
 'dedicated agricultural vehicle': 0.5323412698412698,
 'enclosed_box': 0.6463352536159533,
 'enclosed_cab': 0.2907035429269597,
 'flatbed': 0.4188624338624338,
 'green': 0.04768862342315716,
 'harnessed_to_a_cart': 0.11041046723608668,
 'hatchback': 0.8149043959980377,
 'jeep': 0.12514606086779878,
 'ladder': 0.020864457138074614,
 'large vehicle': 0.967633060324956,
 'light truck': 0.4683170571454602,
 'luggage_carrier': 0.8006216630324111,
 'minibus': 0.360375903861456,
 'minivan': 0.4481674534599078,
 'open_cargo_area': 0.7680517560320581,
 'other': 0.10624473746628872,
 'pickup': 0.63983575751303,
 'prime mover': 0.664743480262804,
 'red': 0.8395585070336768,
 'sedan': 0.9608849383502771,
 'silver/grey': 0.806923012930991,
 'small vehicle': 0.9992398359940191,
 'soft_shell_box': 0.1234974359

In [8]:
sorted(scores.items(), key = lambda x: x[1], reverse=True)

[('small vehicle', 0.9992398359940191),
 ('large vehicle', 0.967633060324956),
 ('bus', 0.9654211140027441),
 ('ac_vents', 0.9635451522157912),
 ('sedan', 0.9608849383502771),
 ('white', 0.9312070194783544),
 ('yellow', 0.8958952996676354),
 ('black', 0.8741999349632973),
 ('sunroof', 0.8697092472193055),
 ('red', 0.8395585070336768),
 ('hatchback', 0.8149043959980377),
 ('silver/grey', 0.806923012930991),
 ('luggage_carrier', 0.8006216630324111),
 ('open_cargo_area', 0.7680517560320581),
 ('van', 0.7482196008475951),
 ('crane truck', 0.732905982905983),
 ('prime mover', 0.664743480262804),
 ('enclosed_box', 0.6463352536159533),
 ('pickup', 0.63983575751303),
 ('blue', 0.6088142478285306),
 ('truck', 0.5562847704333055),
 ('dedicated agricultural vehicle', 0.5323412698412698),
 ('light truck', 0.4683170571454602),
 ('minivan', 0.4481674534599078),
 ('flatbed', 0.4188624338624338),
 ('minibus', 0.360375903861456),
 ('wrecked', 0.30877492983981797),
 ('enclosed_cab', 0.2907035429269597),